# **Homework 10 - Adversarial Attack**

Slides: https://reurl.cc/v5kXkk

Video(En): https://youtu.be/313TZsUDQ48

Video(Zh): https://youtu.be/xWHpPEvkDiE

TA: ntu-ml-2021spring-ta@googlegroups.com

## Enviroment & Download

We make use of [pytorchcv](https://pypi.org/project/pytorchcv/) to obtain CIFAR-10 pretrained model, so we need to set up the enviroment first. We also need to download the data (200 images) which we want to attack.

In [1]:
import torch

torch.cuda.get_device_name()

'Tesla V100-SXM2-16GB'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
!ps -aux|grep python

root          60  0.0  0.2 198780 65672 ?        Sl   04:34   0:09 /usr/bin/python2 /usr/local/bin/jupyter-notebook --ip="172.28.0.2" --port=9000 --FileContentsManager.root_dir="/" --LargeFileManager.delete_to_trash=False --MappingKernelManager.root_dir="/content"
root          95  0.1  0.0      0     0 ?        Z    04:36   0:13 [python3] <defunct>
root         256  0.0  0.0  18384  2892 ?        S    04:36   0:00 bash -c tail -n +0 -F "/root/.config/Google/DriveFS/Logs/drive_fs.txt" | python3 /opt/google/drive/drive-filter.py > "/root/.config/Google/DriveFS/Logs/timeouts.txt" 
root         258  0.0  0.0  31744  9668 ?        S    04:36   0:00 python3 /opt/google/drive/drive-filter.py
root        1003  3.2 11.9 26106464 3182472 ?    Ssl  05:51   3:14 /usr/bin/python3 -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-928861c2-d21b-4d9e-9c10-8512a350bbb4.json
root        1023  0.2  0.0 128416 16160 ?        Sl   05:51   0:17 /usr/bin/python3 /usr/local/lib/python3.7/dis

In [ ]:
!kill -9 1003

In [39]:
# set up environment
!pip install pytorchcv

# download
!gdown --id 1fHi1ko7wr80wXkXpqpqpOxuYH1mClXoX -O data.zip

# unzip
!unzip ./data.zip
!rm ./data.zip

Downloading...
From: https://drive.google.com/uc?id=1fHi1ko7wr80wXkXpqpqpOxuYH1mClXoX
To: /data.zip
100% 490k/490k [00:00<00:00, 65.7MB/s]
Archive:  ./data.zip
   creating: data/
   creating: data/deer/
 extracting: data/deer/deer13.png    
 extracting: data/deer/deer6.png     
 extracting: data/deer/deer11.png    
 extracting: data/deer/deer2.png     
 extracting: data/deer/deer10.png    
 extracting: data/deer/deer16.png    
 extracting: data/deer/deer9.png     
 extracting: data/deer/deer20.png    
 extracting: data/deer/deer15.png    
 extracting: data/deer/deer19.png    
 extracting: data/deer/deer5.png     
 extracting: data/deer/deer14.png    
 extracting: data/deer/deer4.png     
 extracting: data/deer/deer8.png     
 extracting: data/deer/deer12.png    
 extracting: data/deer/deer1.png     
 extracting: data/deer/deer7.png     
 extracting: data/deer/deer17.png    
 extracting: data/deer/deer18.png    
 extracting: data/deer/deer3.png     
   creating: data/horse/
 extracting:

## Global Settings

* $\epsilon$ is fixed to be 8. But on **Data section**, we will first apply transforms on raw pixel value (0-255 scale) **by ToTensor (to 0-1 scale)** and then **Normalize (subtract mean divide std)**. $\epsilon$ should be set to $\frac{8}{255 * std}$ during attack.

* Explaination (optional)
    * Denote the first pixel of original image as $p$, and the first pixel of adversarial image as $a$.
    * The $\epsilon$ constraints tell us $\left| p-a \right| <= 8$.
    * ToTensor() can be seen as a function where $T(x) = x/255$.
    * Normalize() can be seen as a function where $N(x) = (x-mean)/std$ where $mean$ and $std$ are constants.
    * After applying ToTensor() and Normalize() on $p$ and $a$, the constraint becomes $\left| N(T(p))-N(T(a)) \right| = \left| \frac{\frac{p}{255}-mean}{std}-\frac{\frac{a}{255}-mean}{std} \right| = \frac{1}{255 * std} \left| p-a \right| <= \frac{8}{255 * std}.$
    * So, we should set $\epsilon$ to be $\frac{8}{255 * std}$ after ToTensor() and Normalize().

In [23]:
import torch
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
batch_size = 8

# the mean and std are the calculated statistics from cifar_10 dataset
cifar_10_mean = (0.491, 0.482, 0.447) # mean for the three channels of cifar_10 images
cifar_10_std = (0.202, 0.199, 0.201) # std for the three channels of cifar_10 images

# convert mean and std to 3-dimensional tensors for future operations
mean = torch.tensor(cifar_10_mean).to(device).view(3, 1, 1)
std = torch.tensor(cifar_10_std).to(device).view(3, 1, 1)

epsilon = 8/255/std
# TODO: iterative fgsm attack
# alpha (step size) can be decided by yourself
alpha = 0.8/255/std + 0.03
print(alpha)
root = './data' # directory for storing benign images
# benign images: images which do not contain adversarial perturbations
# adversarial images: images which include adversarial perturbations

cuda
tensor([[[0.0455]],

        [[0.0458]],

        [[0.0456]]], device='cuda:0')


## Data

Construct dataset and dataloader from root directory. Note that we store the filename of each image for future usage.

In [4]:
import os
import glob
import shutil
import numpy as np
from PIL import Image
from torchvision.transforms import transforms
from torch.utils.data import Dataset, DataLoader

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(cifar_10_mean, cifar_10_std)
])

class AdvDataset(Dataset):
    def __init__(self, data_dir, transform):
        self.images = []
        self.labels = []
        self.names = []
        '''
        data_dir
        ├── class_dir
        │   ├── class1.png
        │   ├── ...
        │   ├── class20.png
        '''
        for i, class_dir in enumerate(sorted(glob.glob(f'{data_dir}/*'))):
            images = sorted(glob.glob(f'{class_dir}/*'))
            self.images += images
            self.labels += ([i] * len(images))
            self.names += [os.path.relpath(imgs, data_dir) for imgs in images]
        self.transform = transform
    def __getitem__(self, idx):
        image = self.transform(Image.open(self.images[idx]))
        label = self.labels[idx]
        return image, label
    def __getname__(self):
        return self.names
    def __len__(self):
        return len(self.images)

adv_set = AdvDataset(root, transform=transform)
adv_names = adv_set.__getname__()
adv_loader = DataLoader(adv_set, batch_size=batch_size, shuffle=False)

print(f'number of images = {adv_set.__len__()}')

number of images = 200


## Utils -- Benign Images Evaluation

In [5]:
# to evaluate the performance of model on benign images
def epoch_benign(model, loader, loss_fn):
    model.eval()
    train_acc, train_loss = 0.0, 0.0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        yp = model(x)
        loss = loss_fn(yp, y)
        train_acc += (yp.argmax(dim=1) == y).sum().item()
        train_loss += loss.item() * x.shape[0]
    return train_acc / len(loader.dataset), train_loss / len(loader.dataset)

## Utils -- Attack Algorithm

In [33]:
# perform fgsm attack
def fgsm(model, x, y, loss_fn, epsilon=epsilon):
    x_adv = x.detach().clone() # initialize x_adv as original benign image x
    x_adv.requires_grad = True # need to obtain gradient of x_adv, thus set required grad
    
    # ave_log = 0
    # for i in model:
    #   log = i(x_adv)
    #   ave_log += log
    # ave_log = ave_log/len(model)
    # #ave_log = model(x_adv)
    # loss = loss_fn(ave_log, y) # calculate loss
    # print(loss)
    # loss.backward()

    
    ave_log = model(x_adv)
    loss = loss_fn(ave_log, y) # calculate loss
    print(loss)
    loss.backward() # calculate gradient
    
    # fgsm: use gradient ascent on x_adv to maximize loss
    x_adv = x_adv + epsilon * x_adv.grad.detach().sign() 
    # x_adv.grad.detach().sign() -> sign(∇xJ(x, y))
    return x_adv

# TODO: perform iterative fgsm attack
# set alpha as the step size in Global Settings section
# alpha and num_iter can be decided by yourself
def ifgsm(model, x, y, loss_fn, epsilon=epsilon, alpha=alpha, num_iter=20):
    # initialize x_adv as original benign image x
    x_adv = x.detach().clone()
    # write a loop of num_iter to represent the iterative times
    # for each loop
    for i in range(num_iter):
        # call fgsm with (epsilon = alpha) to obtain new x_adv
        x_adv = fgsm(model, x_adv, y, loss_fn, alpha)
        # clip new x_adv back to [x-epsilon, x+epsilon]
        x_adv = torch.max(torch.min(x_adv, x+epsilon), x-epsilon)
    return x_adv

g=0
mu=0.9
def mf(model, x, y, loss_fn, epsilon=epsilon, gg=g, m=mu):
    x_adv = x.detach().clone() # initialize x_adv as original benign image x
    x_adv.requires_grad = True # need to obtain gradient of x_adv, thus set required grad
    ave_log = 0

    for i in model:
      log = i(x_adv)
      ave_log += log
    #   loss = loss_fn(i(x_adv), y)
    #   ave_log += loss
    ave_log = ave_log/len(model)
    loss = loss_fn(ave_log, y)
    # ave_log = model(x_adv)
     # calculate loss
    print(loss)
    loss.backward() # calculate gradient
    # fgsm: use gradient ascent on x_adv to maximize loss
    gg = mu*gg + x_adv.grad.detach()
    x_adv = x_adv + epsilon * gg.sign()
    # x_adv.grad.detach().sign() -> sign(∇xJ(x, y))
    return x_adv

# https://arxiv.org/pdf/1710.06081.pdf

def mifgsm(model, x, y, loss_fn, epsilon=epsilon, alpha=alpha, num_iter=60):
    # initialize x_adv as original benign image x
    x_adv = x.detach().clone() # initialize x_adv as original benign image x
    # write a loop of num_iter to represent the iterative times
    # for each loop

    for i in range(num_iter):
        # x_adv = x.detach().clone()
        # x_adv.requires_grad = True # need to obtain gradient of x_adv, thus set required grad
        # loss = loss_fn(model(x_adv), y) # calculate loss
        # loss.backward()
        # g = mu*g + x_adv.grad.detach().sign()
        # x_adv = x_adv + alpha * g
        # alpha = 
        # i += 1
        x_adv = mf(model, x_adv, y, loss_fn, alpha)
        x_adv = torch.max(torch.min(x_adv, x+epsilon), x-epsilon)

    return x_adv

## Utils -- Attack

* Recall
    * ToTensor() can be seen as a function where $T(x) = x/255$.
    * Normalize() can be seen as a function where $N(x) = (x-mean)/std$ where $mean$ and $std$ are constants.

* Inverse function
    * Inverse Normalize() can be seen as a function where $N^{-1}(x) = x*std+mean$ where $mean$ and $std$ are constants.
    * Inverse ToTensor() can be seen as a function where $T^{-1}(x) = x*255$.

* Special Noted
    * ToTensor() will also convert the image from shape (height, width, channel) to shape (channel, height, width), so we also need to transpose the shape back to original shape.
    * Since our dataloader samples a batch of data, what we need here is to transpose **(batch_size, channel, height, width)** back to **(batch_size, height, width, channel)** using np.transpose.

In [7]:
# perform adversarial attack and generate adversarial examples
def gen_adv_examples_en(en, model, loader, attack, loss_fn):
    model.eval()
    adv_names = []
    train_acc, train_loss = 0.0, 0.0
    for i, (x, y) in enumerate(loader):
        x, y = x.to(device), y.to(device)
        x_adv = attack(en, x, y, loss_fn) # obtain adversarial examples
        yp = model(x_adv)
        loss = loss_fn(yp, y)
        train_acc += (yp.argmax(dim=1) == y).sum().item()
        train_loss += loss.item() * x.shape[0]
        # store adversarial examples
        adv_ex = ((x_adv) * std + mean).clamp(0, 1) # to 0-1 scale
        adv_ex = (adv_ex * 255).clamp(0, 255) # 0-255 scale
        adv_ex = adv_ex.detach().cpu().data.numpy().round() # round to remove decimal part
        adv_ex = adv_ex.transpose((0, 2, 3, 1)) # transpose (bs, C, H, W) back to (bs, H, W, C)
        adv_examples = adv_ex if i == 0 else np.r_[adv_examples, adv_ex]
    return adv_examples, train_acc / len(loader.dataset), train_loss / len(loader.dataset)

# create directory which stores adversarial examples
def create_dir(data_dir, adv_dir, adv_examples, adv_names):
    if os.path.exists(adv_dir) is not True:
        _ = shutil.copytree(data_dir, adv_dir)
    for example, name in zip(adv_examples, adv_names):
        im = Image.fromarray(example.astype(np.uint8)) # image pixel value should be unsigned int
        im.save(os.path.join(adv_dir, name))

In [47]:
# perform adversarial attack and generate adversarial examples
def gen_adv_examples(model, loader, attack, loss_fn):
    model.eval()
    adv_names = []
    train_acc, train_loss = 0.0, 0.0
    for i, (x, y) in enumerate(loader):
        x, y = x.to(device), y.to(device)
        x_adv = attack(model, x, y, loss_fn) # obtain adversarial examples
        yp = model(x_adv)
        loss = loss_fn(yp, y)
        train_acc += (yp.argmax(dim=1) == y).sum().item()
        train_loss += loss.item() * x.shape[0]
        # store adversarial examples
        adv_ex = ((x_adv) * std + mean).clamp(0, 1) # to 0-1 scale
        adv_ex = (adv_ex * 255).clamp(0, 255) # 0-255 scale
        adv_ex = adv_ex.detach().cpu().data.numpy().round() # round to remove decimal part
        adv_ex = adv_ex.transpose((0, 2, 3, 1)) # transpose (bs, C, H, W) back to (bs, H, W, C)
        adv_examples = adv_ex if i == 0 else np.r_[adv_examples, adv_ex]
    return adv_examples, train_acc / len(loader.dataset), train_loss / len(loader.dataset)

# create directory which stores adversarial examples
def create_dir(data_dir, adv_dir, adv_examples, adv_names):
    if os.path.exists(adv_dir) is not True:
        _ = shutil.copytree(data_dir, adv_dir)
    for example, name in zip(adv_examples, adv_names):
        im = Image.fromarray(example.astype(np.uint8))
        print(im) # image pixel value should be unsigned int
        im.save(os.path.join(adv_dir, name))

In [9]:
# perform adversarial attack and generate adversarial examples
def get_en_loss(model, loader, attack, loss_fn):
    model.eval()
    adv_names = []
    logits_en = []
    train_acc, train_loss = 0.0, 0.0
    for i, (x, y) in enumerate(loader):
        x, y = x.to(device), y.to(device)
        x_adv = attack(model, x, y, loss_fn) # obtain adversarial examples
        logits = model(x_adv)
        logits_en.append(logits)
    return logits_en


## Model / Loss Function

Model list is available [here](https://github.com/osmr/imgclsmob/blob/master/pytorch/pytorchcv/model_provider.py). Please select models which has _cifar10 suffix. Some of the models cannot be accessed/loaded. You can safely skip them since TA's model will not use those kinds of models.

In [10]:
enmodel = ['resnet20_cifar10']
len(enmodel)

1

In [11]:
from pytorchcv.model_provider import get_model as ptcv_get_model
model = ptcv_get_model('resnet20_cifar10', pretrained=True).to(device).eval()
model1 = ptcv_get_model('resnet56_cifar10', pretrained=True).to(device).eval()
model2 = ptcv_get_model('resnet110_cifar10', pretrained=True).to(device).eval()
model3 = ptcv_get_model('resnet164bn_cifar10', pretrained=True).to(device).eval()
model4 = ptcv_get_model('resnet272bn_cifar10', pretrained=True).to(device).eval()
model5 = ptcv_get_model('resnet542bn_cifar10', pretrained=True).to(device).eval()
model6 = ptcv_get_model('resnet1001_cifar10', pretrained=True).to(device).eval()
model7 = ptcv_get_model('resnet1202_cifar10', pretrained=True).to(device).eval()
model8 = ptcv_get_model('preresnet20_cifar10', pretrained=True).to(device).eval()
model9 = ptcv_get_model('preresnet272bn_cifar10', pretrained=True).to(device).eval()
model10 = ptcv_get_model('preresnet1202_cifar10', pretrained=True).to(device).eval()
model11 = ptcv_get_model('preresnet542bn_cifar10', pretrained=True).to(device).eval()


In [12]:
en = [model, model1, model2, model3, model4, model5, model6, model7, model8, model9, model10, model11]

In [34]:
from pytorchcv.model_provider import get_model as ptcv_get_model
model_x = ptcv_get_model('resnet20_cifar10', pretrained=True).to(device)
loss_fn = nn.CrossEntropyLoss()
adv_examples, acc, loss = gen_adv_examples_en(en, model_x, adv_loader, mifgsm, loss_fn)
print(f'fgsm_acc = {acc:.5f}, fgsm_loss = {loss:.5f}')


tensor(0.0505, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.8100, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.8448, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(6.3337, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(7.4751, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(8.3070, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(8.7892, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(9.5123, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.0581, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.4125, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.7887, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(11.3481, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(11.2709, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(11.7872, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(11.7945, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(12.2940, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(12.3402, device='cuda:0', grad_fn=<NllLos

In [ ]:
create_dir(root, 'mifgsm', adv_examples, adv_names)

In [51]:
for example, name in zip(adv_examples, adv_names):
    im = Image.fromarray(example.astype(np.uint8))
    im.save(os.path.join('mifgsm', name))


In [50]:
from pytorchcv.model_provider import get_model as ptcv_get_model

model = ptcv_get_model('resnet20_cifar10', pretrained=True).to(device)
loss_fn = nn.CrossEntropyLoss()

benign_acc, benign_loss = epoch_benign(model, adv_loader, loss_fn)
print(f'benign_acc = {benign_acc:.5f}, benign_loss = {benign_loss:.5f}')

benign_acc = 0.94500, benign_loss = 0.17499


## FGSM

In [124]:
from pytorchcv.model_provider import get_model as ptcv_get_model

model = ptcv_get_model('resnet20_cifar10', pretrained=True).to(device)
loss_fn = nn.CrossEntropyLoss()
adv_examples, fgsm_acc, fgsm_loss = gen_adv_examples(model, adv_loader, fgsm, loss_fn)
print(f'fgsm_acc = {fgsm_acc:.5f}, fgsm_loss = {fgsm_loss:.5f}')

create_dir(root, 'fgsm', adv_examples, adv_names)

tensor(0.0687, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0134, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0440, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0083, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.4167, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.8183, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0891, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0335, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0724, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.6943, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0624, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.1116, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.4847, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.3297, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0278, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.3480, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0053, device='cuda:0', grad_fn=<NllLossBackward

## I-FGSM

In [191]:
# TODO: iterative fgsm attack

model_x = ptcv_get_model('resnet20_cifar10', pretrained=True).to(device)
adv_examples, ifgsm_acc, ifgsm_loss = gen_adv_examples_en(en[0], model_x,adv_loader, ifgsm, loss_fn)
print(f'ifgsm_acc = {ifgsm_acc:.5f}, ifgsm_loss = {ifgsm_loss:.5f}')

create_dir(root, 'ifgsm', adv_examples, adv_names)

tensor(0.0687, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.7194, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.5506, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(6.7381, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(8.5497, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.2791, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(11.5644, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(12.5144, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(13.2456, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(14.2239, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(14.6999, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(15.3606, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(15.9158, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(16.4164, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(16.4701, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(16.9543, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(17.3837, device='cuda:0', grad_fn=<Nll

ifgsm_acc = 0.00000, ifgsm_loss = 20.60922


## Compress the images

In [52]:
# %cd fgsm
# !tar zcvf ../fgsm.tgz *
# %cd ..

%cd mifgsm
!tar zcvf ../mifgsm.tgz *
%cd ..

/mifgsm
airplane/
airplane/airplane9.png
airplane/airplane16.png
airplane/airplane2.png
airplane/airplane14.png
airplane/airplane19.png
airplane/airplane8.png
airplane/airplane10.png
airplane/airplane15.png
airplane/airplane17.png
airplane/airplane3.png
airplane/airplane6.png
airplane/airplane4.png
airplane/airplane20.png
airplane/airplane7.png
airplane/airplane13.png
airplane/airplane1.png
airplane/airplane18.png
airplane/airplane5.png
airplane/airplane11.png
airplane/airplane12.png
automobile/
automobile/automobile5.png
automobile/automobile6.png
automobile/automobile17.png
automobile/automobile9.png
automobile/automobile11.png
automobile/automobile19.png
automobile/automobile4.png
automobile/automobile14.png
automobile/automobile2.png
automobile/automobile20.png
automobile/automobile7.png
automobile/automobile13.png
automobile/automobile12.png
automobile/automobile8.png
automobile/automobile1.png
automobile/automobile15.png
automobile/automobile18.png
automobile/automobile3.png
auto

## Visualization

In [ ]:
import matplotlib.pyplot as plt

classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10, 20))
cnt = 0
for i, cls_name in enumerate(classes):
    path = f'{cls_name}/{cls_name}1.png'
    # benign image
    cnt += 1
    plt.subplot(len(classes), 4, cnt)
    im = Image.open(f'./data/{path}')
    logit = model(transform(im).unsqueeze(0).to(device))[0]
    predict = logit.argmax(-1).item()
    prob = logit.softmax(-1)[predict].item()
    plt.title(f'benign: {cls_name}1.png\n{classes[predict]}: {prob:.2%}')
    plt.axis('off')
    plt.imshow(np.array(im))
    # adversarial image
    cnt += 1
    plt.subplot(len(classes), 4, cnt)
    im = Image.open(f'./fgsm/{path}')
    logit = model(transform(im).unsqueeze(0).to(device))[0]
    predict = logit.argmax(-1).item()
    prob = logit.softmax(-1)[predict].item()
    plt.title(f'adversarial: {cls_name}1.png\n{classes[predict]}: {prob:.2%}')
    plt.axis('off')
    plt.imshow(np.array(im))
plt.tight_layout()
plt.show()